In [ ]:
#  모듈 불러오기
import tensorflow as tf
import os
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# 디렉토리 확인
os.listdir('../')
os.listdir('../input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data')


In [ ]:
#  디렉토리 저장
train_dir = '../input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/train'
val_dir = '../input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/valid'
test_dir='../input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/test'


In [ ]:
# 데이터 제너레이터 생성
train_datagen = ImageDataGenerator(
    rescale=1. / 255,		# 리스케일링
    shear_range=0.2,		# 시계반대방향으로 밀림 강도를 0 ~ 0.2 사이로 조정
    zoom_range=0.2,		# 원본 이미지 확대 축소 (1-값 ~ 1+값), 0.8 ~ 1.2 로 조정
    horizontal_flip=True)		# 좌/우 뒤집기 ,    vertical_flip = True 상/하 뒤집기
val_datagen = ImageDataGenerator(rescale=1./255)

# 트레인데이터 디렉토리, 배치, 이미지크기 지정
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(200, 200),		# 이미지를 200 X 200으로 조정		
        batch_size=10,			# 배치 사이즈를 10으로 조정
        class_mode='categorical')		# 분류모델로 지정 (생략가능)

# 검증용데이터 디렉토리, 배치, 이미지크기 지정
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(200, 200),
        batch_size=10,
        class_mode='categorical')


In [ ]:
# 모델 생성
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(input_shape=(200,200,3), kernel_size=(3,3), filters=32, padding='same', activation='relu'),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, padding='same', activation='relu'),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=512, activation='relu'),
  tf.keras.layers.Dense(units=256, activation='relu'),
  tf.keras.layers.Dropout(rate=0.5),
  tf.keras.layers.Dense(units=6, activation='softmax')
])

model.summary()  # 모델 확인


In [ ]:
# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [ ]:
# 모델 학습
history = model.fit(train_generator, epochs=20, validation_data=validation_generator)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()  


In [ ]:
# 케라스 모델로 저장
save_path = 'my_model.h5'
model.save(save_path, include_optimizer=True)
